In [2]:
data_file = "../data/drugs_18-May-2021_ATC-N02_must-have-NCT.txt"

In [3]:
fo = open(data_file,'r', encoding="ISO-8859-1")
header = fo.readline()
header = header.strip().split("\t")

In [4]:
header

['Drug ID',
 'Drug Name',
 'CAS Number',
 'Chemical Name',
 'New Molecular Entity',
 'Drug Interactions',
 'Comparative Efficacy',
 'Highest Development Phase',
 'Indication',
 'Molecular Formula',
 'Mechanism Of Action',
 'Organisation',
 'Pharmacodynamic Characteristics',
 'Route Of Administration',
 'Route Of Elimination',
 'Pharmacokinetic Characteristics',
 'Adverse Events',
 'Therapeutic Class (EphMRA)',
 'Therapeutic Class (WHO)',
 'Synonyms/Brand Names',
 'Antimicrobial Activity',
 'Immunogenicity',
 'Last Significant Update Date',
 'Therapeutic Area',
 'Orphan Designation',
 'License Availability',
 'Brands',
 'On Fastrack',
 'Fastrack country',
 'On PRIME',
 'PRIME country',
 'Biomarkers-Inclusion',
 'Biomarkers-Exclusion',
 'Biomarkers-Outcome Measure',
 'Biomarkers-Safety',
 'Biomarkers-not specified',
 'URL']

In [5]:
ind = 8; moa = 10

In [6]:
therapeutic = [header.index(x) for x in header if 'Therapeutic' in x]
print(therapeutic)

[17, 18, 23]


In [7]:
indicators = []
mechanisms = []
thEs = []
thWs = []
thAs = []

for line in fo:
    line = line.strip().split('\t')
    rowid = line[0]
    try:
        indicator = line[ind]
        mechanism = line[moa]
        thE = line[17]
        thW = line[18]
        thA = line[23]
    except:
        pass
    indicators.append(indicator.split(","))
    mechanisms.append(mechanism.split(","))
    thEs.append(thE.split(","))
    thWs.append(thW.split(","))
    thAs.append(thA.split(","))



In [8]:
indicators_flat = [item.strip('\" ').replace('\'','') for sublist in indicators for item in sublist]
indicators_flat = set(indicators_flat)

In [9]:
mechanisms_flat = [item.strip('\" ').replace('\'','') for sublist in mechanisms for item in sublist]
mechanisms_flat = set(mechanisms_flat)

In [10]:
thE_flat = [item.strip('\" ').replace('\'','') for sublist in thEs for item in sublist]
thE_flat = set(thE_flat)
thW_flat = [item.strip('\" ').replace('\'','') for sublist in thWs for item in sublist]
thW_flat = set(thW_flat)
thA_flat = [item.strip('\" ').replace('\'','') for sublist in thAs for item in sublist]
thA_flat = set(thA_flat)

In [11]:
len(mechanisms_flat)

323

In [12]:
len(indicators_flat)

372

In [13]:
print(len(thE_flat))
print(len(thW_flat))
print(len(thA_flat))

197
303
47


In [14]:
import pandas as pd

In [15]:
df = pd.read_table('../data/drugs_18-May-2021_ATC-N02_must-have-NCT.txt', encoding="ISO-8859-1")

In [16]:
df.fillna('', inplace=True)

In [17]:
df['Therapeutic Class (EphMRA)']

0      N2 (Analgesics), N2C (Anti-Migraine Preparatio...
1            N2C9 (All other anti-migraine preparations)
2      C1 (Cardiac Therapy), C10 (Lipid-Regulating/An...
3                                        N2 (Analgesics)
4      A8 (Antiobesity Preparations, Excluding Dietet...
                             ...                        
523    C6A (Other Cardiovascular Products), M1 (Anti-...
524                                      N2 (Analgesics)
525    A10X (Other Drugs Used in Diabetes), G4D4 (Uri...
526                                      N2 (Analgesics)
527    A4A9 (Other antiemetics and antinauseants), A7...
Name: Therapeutic Class (EphMRA), Length: 528, dtype: object

In [18]:
cleaned_indication = df.set_index('Drug ID').Indication.str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Indication')

In [19]:
cleaned_indication.Indication = cleaned_indication.Indication.str.strip(" ")

In [20]:
pd.set_option("min_rows", 20)
cleaned_indication.Indication.value_counts().head(20)

Pain                         185
Neuropathic pain             127
Migraine                      77
Postoperative pain            62
Diabetic neuropathies         46
Cancer pain                   38
Musculoskeletal pain          37
Postherpetic neuralgia        32
Back pain                     29
Epilepsy                      28
Acute pain                    23
Fibromyalgia                  23
Major depressive disorder     22
Inflammation                  20
Rheumatoid arthritis          19
Dental pain                   18
Alzheimer's disease           17
Osteoarthritis                14
Pruritus                      13
Parkinson's disease           13
Name: Indication, dtype: int64

In [21]:
ind = pd.get_dummies(cleaned_indication, prefix='IND').groupby(level=0).sum()

In [22]:
cleaned_mechanism = df.set_index('Drug ID')['Mechanism Of Action'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Mechanism')
cleaned_mechanism.Mechanism = cleaned_mechanism.Mechanism.str.strip(" ")
mch = pd.get_dummies(cleaned_mechanism, prefix='MCH').groupby(level=0).sum()

In [23]:
cleaned_therape = df.set_index('Drug ID')['Therapeutic Class (EphMRA)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapE')
cleaned_therape.TherapE = cleaned_therape.TherapE.str.strip(" ")
the = pd.get_dummies(cleaned_therape, prefix='THE').groupby(level=0).sum()

In [24]:
cleaned_therape.TherapE.value_counts()

N2 (Analgesics)                                       308
N2B (Non-Narcotics and Anti-Pyretics)                  96
N2A (Narcotics)                                        70
N2C (Anti-Migraine Preparations)                       52
M1 (Anti-Inflammatory and Anti-Rheumatic Products)     50
N7 (Other CNS Drugs)                                   42
N7X (All other CNS drugs)                              36
N3A (Anti-Epileptics)                                  35
N6A (Anti-Depressants and Mood Stabilisers)            25
L1 (Antineoplastics)                                   21
                                                     ... 
D11 (Other Dermatological Preparations)                 1
H1C3 (Antigonadotrophin-releasing hormones)             1
B3X (Other Anti-Anaemic Products                        1
D5A (Topical Antipsoriasis Products)                    1
B1C5 (Platelet aggregation inhibitors                   1
T1X (Other imaging agents)                              1
Folinic Acid) 

In [25]:
cleaned_therapw = df.set_index('Drug ID')['Therapeutic Class (WHO)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapW')
cleaned_therapw.TherapW = cleaned_therapw.TherapW.str.strip(" ")
thw = pd.get_dummies(cleaned_therapw, prefix='THW').groupby(level=0).sum()

In [26]:
cleaned_therapw.TherapW.value_counts()

N02 (Analgesics)                                                                283
N02B (Other Analgesics and Antipyretics)                                         96
N07 (Other Nervous System Drugs)                                                 36
N02C (Antimigraine Preparations)                                                 36
M01 (Antiinflammatory and Antirheumatic Products)                                36
N07X (Other Nervous System Drugs)                                                35
L01 (Antineoplastic Agents)                                                      21
A10X (Other Drugs Used in Diabetes)                                              20
N02C-X (Other antimigraine preparations)                                         18
N02A (Opioids)                                                                   17
                                                                               ... 
D05A-X (Other antipsoriatics for topical use)                               

In [27]:
cleaned_therapa = df.set_index('Drug ID')['Therapeutic Area'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapA')
cleaned_therapa.TherapA = cleaned_therapa.TherapA.str.strip(" ")
tha = pd.get_dummies(cleaned_therapa, prefix='THA').groupby(level=0).sum()

In [28]:
cleaned_therapa.TherapA.value_counts()

Pain                                                434
Advances in the Treatment of Nausea and Migraine     83
Neurological Disorders                               69
Rheumatic Disease                                    62
Affective Disorders                                  46
Diabetes                                             44
Skin Disorders                                       35
Drug Delivery Systems                                33
Epilepsy and Seizure Disorders                       32
Cancer                                               31
Anxiety Disorders                                    30
Musculoskeletal Disorders                            30
Parkinson's Disease and Movement Disorders           27
Respiratory Tract Disorders                          27
Genitourinary Disorders                              24
Inflammation                                         24
Drug Withdrawal                                      22
Obstructive Airways Disease                     

In [29]:
df = df.set_index('Drug ID').join(ind).join(mch).join(the).join(thw).join(tha)

In [30]:
df.to_csv('../data/drugs_may_20.txt',sep="\t")

In [31]:
ind.shape

(528, 366)

In [32]:
mch.shape

(528, 322)

# Market-Basket Analysis

In [32]:
#!conda install MLxtend --channel conda-forge --yes

In [33]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [34]:
frequent_indicators = apriori(ind, min_support=0.01, use_colnames=True, max_len=4)

In [35]:
frequent_indicators.sort_values(by=['support'],ascending=False)

,support,itemsets
39,0.350379,(IND_Pain)
34,0.240530,(IND_Neuropathic pain)
30,0.145833,(IND_Migraine)
43,0.117424,(IND_Postoperative pain)
19,0.087121,(IND_Diabetic neuropathies)
13,0.071970,(IND_Cancer pain)
33,0.070076,(IND_Musculoskeletal pain)
42,0.060606,(IND_Postherpetic neuralgia)
81,0.056818,"(IND_Neuropathic pain, IND_Pain)"
8,0.054924,(IND_Back pain)


In [36]:
rules = association_rules(frequent_indicators, metric="lift", min_threshold=1)


In [37]:
rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,(IND_Lennox-Gastaut syndrome),(IND_Epilepsy),0.013258,0.053030,0.011364,0.857143,16.163265,0.010661,6.628788
49,(IND_Partial epilepsies),(IND_Neuropathic pain),0.017045,0.240530,0.013258,0.777778,3.233596,0.009158,3.417614
20,(IND_Cluster headache),(IND_Migraine),0.015152,0.145833,0.011364,0.750000,5.142857,0.009154,3.416667
62,(IND_Stroke),(IND_Pain),0.015152,0.350379,0.011364,0.750000,2.140541,0.006055,2.598485
76,"(IND_Postoperative pain, IND_Neuropathic pain)",(IND_Pain),0.022727,0.350379,0.017045,0.750000,2.140541,0.009082,2.598485
37,(IND_Partial epilepsies),(IND_Epilepsy),0.017045,0.053030,0.011364,0.666667,12.571429,0.010460,2.840909
5,(IND_Amyotrophic lateral sclerosis),(IND_Multiple sclerosis),0.018939,0.022727,0.011364,0.600000,26.400000,0.010933,2.443182
42,(IND_Major depressive disorder),(IND_Neuropathic pain),0.041667,0.240530,0.022727,0.545455,2.267717,0.012705,1.670833
71,"(IND_Migraine, IND_Epilepsy)",(IND_Neuropathic pain),0.022727,0.240530,0.011364,0.500000,2.078740,0.005897,1.518939
70,"(IND_Migraine, IND_Neuropathic pain)",(IND_Epilepsy),0.022727,0.053030,0.011364,0.500000,9.428571,0.010158,1.893939


In [38]:
frequent_mechanisms = apriori(mch, min_support=0.01, use_colnames=True, max_len=4)
frequent_mechanisms.sort_values(by=['support'],ascending=False)

,support,itemsets
20,0.111742,(MCH_Opioid mu receptor agonists)
12,0.062500,(MCH_Cyclooxygenase inhibitors)
29,0.053030,(MCH_Undefined mechanism)
25,0.049242,(MCH_Serotonin uptake inhibitors)
18,0.039773,(MCH_Opioid kappa receptor agonists)
26,0.037879,(MCH_Sodium channel antagonists)
11,0.035985,(MCH_Cyclo-oxygenase 2 inhibitors)
21,0.032197,(MCH_Opioid receptor agonists)
16,0.032197,(MCH_Opioid delta receptor agonists)
3,0.030303,(MCH_Adrenergic uptake inhibitors)


In [39]:
rules_mch = association_rules(frequent_mechanisms, metric="lift", min_threshold=1)
rules_mch[ (rules_mch['lift'] >= 2) &
       (rules_mch['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
35,(MCH_Opioid delta receptor antagonists),"(MCH_Opioid kappa receptor antagonists, MCH_No...",0.013258,0.013258,0.013258,1.000000,75.428571,0.013082,inf
38,"(MCH_Opioid delta receptor antagonists, MCH_Op...",(MCH_Nociceptin receptor agonists),0.011364,0.018939,0.011364,1.000000,52.800000,0.011148,inf
29,(MCH_Serotonin 1B receptor agonists),(MCH_Serotonin 1D receptor agonists),0.024621,0.030303,0.024621,1.000000,33.000000,0.023875,inf
30,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.013258,0.013258,0.013258,1.000000,75.428571,0.013082,inf
31,"(MCH_Opioid kappa receptor antagonists, MCH_Op...",(MCH_Nociceptin receptor agonists),0.013258,0.018939,0.013258,1.000000,52.800000,0.013006,inf
61,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
62,"(MCH_Opioid kappa receptor antagonists, MCH_Op...",(MCH_Nociceptin receptor agonists),0.011364,0.018939,0.011364,1.000000,52.800000,0.011148,inf
32,"(MCH_Nociceptin receptor agonists, MCH_Opioid ...",(MCH_Opioid kappa receptor antagonists),0.013258,0.015152,0.013258,1.000000,66.000000,0.013057,inf
17,(MCH_Opioid delta receptor antagonists),(MCH_Opioid kappa receptor antagonists),0.013258,0.015152,0.013258,1.000000,66.000000,0.013057,inf
63,"(MCH_Nociceptin receptor agonists, MCH_Opioid ...",(MCH_Opioid kappa receptor antagonists),0.011364,0.015152,0.011364,1.000000,66.000000,0.011191,inf


In [40]:
rules_mch['lhs items'] = [len(x) for x in rules_mch['antecedents']]
rules_mch[rules_mch['lhs items']>1].sort_values('lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,lhs items
30,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.013258,0.013258,0.013258,1.000000,75.428571,0.013082,inf,2
61,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf,3
69,"(MCH_Opioid delta receptor antagonists, MCH_Op...","(MCH_Opioid kappa receptor antagonists, MCH_No...",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf,2
64,"(MCH_Opioid kappa receptor antagonists, MCH_No...","(MCH_Opioid delta receptor antagonists, MCH_Op...",0.013258,0.011364,0.011364,0.857143,75.428571,0.011213,6.920455,2
56,"(MCH_Opioid delta receptor antagonists, MCH_Op...",(MCH_Opioid kappa receptor antagonists),0.011364,0.015152,0.011364,1.000000,66.000000,0.011191,inf,2


In [41]:
df['Drug Name']

Drug ID
800015670                                   Perampanel - Eisai
800028519                   Lasmiditan - Eli Lilly and Company
800023191                    Losmapimod - Fulcrum Therapeutics
800033181                 Mirogabalin - Daiichi Sankyo Company
800039721    Synthetic cannabidiol - Benuvia Therapeutics/R...
800050633                Vocacapsaicin - Concentric Analgesics
800044788    Diclofenac sodium transdermal patch - Hisamits...
800028218                    Difelikefalin - Cara Therapeutics
800042256         Centhaquine - Midwestern University/Pharmazz
800040151                   Esreboxetine - Axsome Therapeutics
                                   ...                        
800023039                                           BVT 115959
800031069                                             AZD 6088
800029345                                           LY 2590443
800018832                          Fentanyl sublingual - Sosei
800021826                                      

In [42]:
frequent_all = apriori(mch.join(ind).join(tha), min_support=0.01, use_colnames=True, max_len=6)
frequent_all.sort_values(by=['support'],ascending=False)

,support,itemsets
107,0.821970,(THA_Pain)
69,0.350379,(IND_Pain)
362,0.350379,"(THA_Pain, IND_Pain)"
64,0.240530,(IND_Neuropathic pain)
329,0.240530,"(THA_Pain, IND_Neuropathic pain)"
82,0.157197,(THA_Advances in the Treatment of Nausea and M...
60,0.145833,(IND_Migraine)
293,0.142045,(THA_Advances in the Treatment of Nausea and M...
104,0.130682,(THA_Neurological Disorders)
112,0.117424,(THA_Rheumatic Disease)


In [43]:
rules_all = association_rules(frequent_all, metric="lift", min_threshold=1)
rules_all[ (rules_all['lift'] >= 2) &
       (rules_all['confidence'] >= 0.4) ].sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
750,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.013258,0.013258,0.013258,1.000000,75.428571,0.013082,inf
4313,"(IND_Neuropathic pain, THA_Advances in the Tre...","(THA_Pain, IND_Migraine, IND_Epilepsy)",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
2457,"(MCH_Opioid delta receptor antagonists, MCH_Op...","(MCH_Opioid kappa receptor antagonists, MCH_No...",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
755,(MCH_Opioid delta receptor antagonists),"(MCH_Opioid kappa receptor antagonists, MCH_No...",0.013258,0.013258,0.013258,1.000000,75.428571,0.013082,inf
4309,"(IND_Neuropathic pain, IND_Migraine, THA_Epile...","(THA_Pain, THA_Advances in the Treatment of Na...",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
2449,"(MCH_Opioid kappa receptor antagonists, MCH_No...",(MCH_Opioid delta receptor antagonists),0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
4318,(THA_Advances in the Treatment of Nausea and M...,"(THA_Pain, IND_Migraine, THA_Epilepsy and Seiz...",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
4315,"(IND_Migraine, IND_Neuropathic pain, IND_Epile...","(THA_Pain, THA_Advances in the Treatment of Na...",0.011364,0.013258,0.011364,1.000000,75.428571,0.011213,inf
4302,"(THA_Pain, THA_Advances in the Treatment of Na...","(IND_Migraine, IND_Neuropathic pain, IND_Epile...",0.013258,0.011364,0.011364,0.857143,75.428571,0.011213,6.920455
4304,"(THA_Pain, IND_Migraine, IND_Epilepsy)","(IND_Neuropathic pain, THA_Advances in the Tre...",0.013258,0.011364,0.011364,0.857143,75.428571,0.011213,6.920455


In [44]:
df['Adverse Events'].value_counts()

                                                                                                                                                                                                                                                                                                                                    413
Application site reactions                                                                                                                                                                                                                                                                                                            3
Dizziness, Somnolence                                                                                                                                                                                                                                                                                                                 3
Headache, Nausea

In [45]:
df[df['Adverse Events'].str.contains("Aggression")]

,Drug Name,CAS Number,Chemical Name,New Molecular Entity,Drug Interactions,Comparative Efficacy,Highest Development Phase,Indication,Molecular Formula,Mechanism Of Action,...,THA_Peptic Ulcer Disease,THA_Psychotic Disorders,THA_Respiratory Tract Disorders,THA_Rheumatic Disease,THA_Skin Disorders,THA_Thromboses,THA_Transplant Rejection,THA_Vascular Disorders,THA_Viral Infections,THA_Women's Health
Drug ID,,,,,,,,,,,,,,,,,,,,,
800015670,Perampanel - Eisai,380917-97-5,"Benzonitrile, 2-[1',6'-dihydro-6'-oxo-1'-pheny...",Yes,,,Marketed,"Tonic-clonic epilepsy, Partial epilepsies, Sei...",C23 H15 N3 O,AMPA receptor antagonists,...,0,0,0,0,0,0,0,0,0,0
800002881,Vigabatrin - sanofi-aventis,60643-86-9,4-amino-5-hexenoic acid,Yes,,,Marketed,"Epilepsy, Infantile spasms, Complex partial ep...",C6 H11 N O2,4-aminobutyrate transaminase inhibitors,...,0,0,0,0,0,0,0,0,0,0
800000568,Venlafaxine - Pfizer,93413-69-5,"Cyclohexanol, 1-[2-(dimethylamino)-1-(4-methox...",Yes,,"³ Fluoxetine, &gt; Imipramine, ³ Trazodone",Marketed,"Major depressive disorder, Social phobia, Pani...",C17 H27 N O2,"Adrenergic uptake inhibitors, Serotonin uptake...",...,0,0,0,0,0,0,0,0,0,0
